Для анализа были выбраны три романа Л.Н.Толстого - все тома "Войны и мира", "Воскресение" и "Анна Каренина". 

  В качестве частотного слова русского языка я решила вять слово "дело", так как оно часто в употреблении - может, в те времена даже больше, чем сейчас; к тому же используется во всех охватываемых автором сословиях: "дело" может быть юридическим термином, бытовым или даже военным. Сперва анализ проводится для леммы "дело", то есть без учёта конкретной словоформы, затем - для формы "дел" (мне показалось, что таким образом отсеятся некоторые бытовые значения/выражения по типу "не твоё дело" и т.п.).
  
  Использованные меры ассоциации - коэффицент взаимной информации и t-score.

In [19]:
import requests
import html5lib
import subprocess
import codecs
import pickle
import string
import sys
import os

from nltk.tag import pos_tag
from nltk import wordpunct_tokenize
from collections import Counter
from math import log
from os.path import join
from html2text import html2text

In [36]:
import pymorphy2
from string import digits
morph = pymorphy2.MorphAnalyzer()
import math
import pandas as pd
import collections

## Загрузка текста, токенизация, определене POS

In [20]:
# берем страницу html
def get_html(url):
    session = requests.session()
    html = html5lib.parse(
        session.get(url).text,
        treebuilder="lxml",
        namespaceHTMLElements=False,
    ).getroot()
    return html

In [21]:
def get_content(url):
    html = requests.get(url).content
    return html2text(html.decode('cp1251'))

In [22]:
# либо берем данные из файла, либо получаем сами и потом сохраняем в файл
def save_and_get_data(fname, callback):
    try:
        with open(fname, 'rb') as bf:
            data = pickle.load(bf)
    except IOError as ex:
        data = callback()
        with open(fname, 'wb') as bf:
            pickle.dump(data, bf)
    return data



In [23]:
def save_result(cnt_obj, fname):
    with codecs.open(fname, encoding='utf-8', mode='w') as f:
        for elem in cnt_obj.most_common():
            f.write(u"%s %s %s\n" % (elem[0][0], elem[0][1], elem[1]))



In [24]:
# Сохраняем просто текст в unicode
def save_text(text, fname):
    with codecs.open(fname, encoding='utf-8', mode='w') as f:
        f.write(text)

In [25]:
def get_text(urls_xpaths, fname):
    def parse():
        text = ''
        for url in urls_xpaths:
            try:
                t = get_content(url)
            except Exception as ex:
                print (ex)
            text += t
        return text

    data = save_and_get_data(fname, parse)
    return data

In [30]:
#Делим текст на отдельные составляющие
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    remove_digits = str.maketrans('', '', digits)
    s = s.translate(remove_digits)
    t = s.split()
    
    return t

In [32]:
#Определяем части речи каждого слова
def POS(list):
    a=[]
    for i in list:
        a.append(morph.parse(i)[0].tag.POS)
    return a


In [33]:
#Ищем n-граммы
def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

In [50]:
url_xpath_tolstoy = (
        # 4 тома "Война и Мир"
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml',
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml',
        # Анна Каренина
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0080.shtml',
        # Воскресение
        'http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0090.shtml',
    )


In [52]:
text_list=f_tokenizer(text)
#Составляем словарь частей речи
dictionary=dict (zip(text_list,POS(text_list)))


In [51]:
name_dir = 'tolstoy_results'
text = get_text(url_xpath_tolstoy, join(name_dir, 'text.pkl'))
save_text(text, join(name_dir, 'text.txt'))

## Для леммы "дело"

In [86]:
def find_colloc(bigrams):
    colloc=[]
    for i in bigrams:
        if (dictionary[i[0]]=='VERB' and dictionary[i[1]]=='NOUN' and i[1] in lemmas) or (dictionary[i[0]]=='ADJF' and dictionary[i[1]]=='NOUN' and i[1] in lemmas):
            colloc.append(i)
    return colloc

In [122]:
#Посчитать частоту вхождения нормальной формы каждого слова
def freq(text):
    count = collections.Counter()
    for words in text:
        count[morph.parse(words)[0].normal_form] += 1
    return count

In [202]:
#MI measure
def mi(pair):
    MI=math.log(list_count_collocs[pair]*len(text)/(words_freq_count[pair[0]]*words_freq_count[pair[1]]),2)
    if MI>1:
        return MI
    else:
        return 0

In [188]:
#t-score measure
def t_score(pair):
    t_sc=(list_count_collocs[pair]-(words_freq_count[pair[0]]*words_freq_count[pair[1]])/len(text))/(math.sqrt(list_count_collocs[pair]))
    return t_sc

In [53]:
#Находим биграммы, среди них - коллокации (с нормальными формами слов), считаем частоту вхождения каждой нормальной формы слова по тексту.
bigrams=find_ngrams(text_list, 2)
collocs=find_colloc(bigrams)
words_freq_count=freq(text_list)

In [79]:
word='дело'
delo = morph.parse(word)[0]
lemmas=[delo.lexeme[i].word for i in range (len(delo.lexeme))]


In [129]:
#нормируем коллокации по нормальным формам
collocs_normed=[(morph.parse(i[0])[0].normal_form,  morph.parse(i[1])[0].normal_form) for i in collocs]

In [322]:
#частота встречания коллокаций, совпадабщих по нормам
c = collections.Counter()
list_count_collocs=Counter(collocs_normed)
list_count_collocs_form=Counter(collocs_form)

In [278]:
coll_mi={}
coll_t={}
for i in list(list_count_collocs.keys()):
    coll_mi[i]=mi(i)
    coll_t[i]=t_score(i)

In [304]:
h1=pd.DataFrame.from_dict(coll_mi,orient='index').rename(columns={0:'MI'}).reset_index()
h2=pd.DataFrame.from_dict(coll_t,orient='index').rename(columns={0:'t-score'}).reset_index()
h3 = pd.merge(h1,h2,how='inner',on='index')
h3=h3.sort_values(['MI'], ascending=False)
h3.head(20)

,index,MI,t-score
72,"(общежитейский, дело)",11.830388,0.999725
126,"(повестись, дело)",11.830388,0.999725
89,"(душеспасительный, дело)",11.830388,0.999725
176,"(ариергардный, дело)",11.830388,0.999725
139,"(островненский, дело)",11.830388,1.413825
101,"(каменьщический, дело)",11.830388,0.999725
1,"(денежный, дело)",11.119894,3.315134
98,"(шенграбенский, дело)",11.093422,1.731258
66,"(капитальный, дело)",10.830388,0.999451
65,"(бедовый, дело)",10.830388,0.999451


In [305]:
h4=h3.sort_values(['t-score'], ascending=False)
h4.head(20)

,index,MI,t-score
130,"(свой, дело)",5.044179,7.125755
121,"(весь, дело)",4.569091,6.973416
35,"(самый, дело)",6.288494,5.756356
174,"(другой, дело)",5.274465,4.671926
90,"(чистый, дело)",8.733526,4.232673
86,"(общий, дело)",7.370956,4.098197
163,"(ваш, дело)",6.346098,4.072423
46,"(мой, дело)",5.684231,4.042919
137,"(важный, дело)",7.596768,3.979336
138,"(тот, дело)",3.497821,3.973041


## Словоформа "дел"

In [343]:
def find_colloc_form(bigrams):
    colloc=[]
    for i in bigrams:
        if (dictionary[i[0]]=='VERB' and dictionary[i[1]]=='NOUN' and i[1]=='дел') or (dictionary[i[0]]=='ADJF' and dictionary[i[1]]=='NOUN' and i[1]=='дел'):
            colloc.append(i)
    return colloc



In [344]:
collocs_form=find_colloc_form(bigrams)
words_freq_count_form=Counter(text_list)

In [347]:
#MI measure
def mi_form(pair):
    MI=math.log(list_count_collocs_form[pair]*len(text)/(words_freq_count_form[pair[0]]*words_freq_count_form[pair[1]]),2)
    if MI>1:
        return MI
    else:
        return 0
    
#t-score measure
def t_score_form(pair):
    t_sc=(list_count_collocs_form[pair]-(words_freq_count_form[pair[0]]*words_freq_count_form[pair[1]])/len(text))/(math.sqrt(list_count_collocs_form[pair]))
    return t_sc
    
coll_mi_form={}
coll_t_form={}
for i in list(list_count_collocs_form.keys()):
    coll_mi_form[i]=mi_form(i)
    coll_t_form[i]=t_score_form(i)

In [349]:
h1_form=pd.DataFrame.from_dict(coll_mi_form,orient='index').rename(columns={0:'MI'}).reset_index()
h2_form=pd.DataFrame.from_dict(coll_t_form,orient='index').rename(columns={0:'t-score'}).reset_index()
h3_form = pd.merge(h1_form,h2_form,how='inner',on='index')
h3_form.sort_values(['t-score'], ascending=False)

,index,MI,t-score
0,"(этих, дел)",8.886927,1.728392
10,"(иностранных, дел)",15.290649,1.414178
1,"(государственных, дел)",13.483294,1.414090
16,"(всех, дел)",7.457759,1.406169
13,"(общежитейских, дел)",16.290649,0.999988
17,"(судейских, дел)",16.290649,0.999988
5,"(торжественных, дел)",14.705687,0.999963
14,"(ужасных, дел)",13.968721,0.999938
9,"(денежных, дел)",13.968721,0.999938
11,"(внутренних, дел)",13.483294,0.999913


## Заключение.

Верхние две таблицы - наиболее часто встречающиеся в коллекции текстов коллокации с леммой "дело", упорядоченные соответственно по мере ассоциации MI и t-score. Результаты совершенно разные: мера взаимной информации выдала больше определений, являющихся "чистыми" прилагательными, эпитетов, отимённых прилагательных. Вполне логично, учитывая редкость встречаемости результатов (ариергардный,шенграбенский), а ведь именно эта переменная стоит в знаменателе выражения для MI.

Что касается t-score, заметно обилие местоимённых, притяжательных прилагательных, частота встречаемости оторых отдельно от слова должна быть явно больше.

Вариант с словоформой "дел" действительно отсёк некоторые коллокативные определения, но это скорее отымённые прилагательные - коллокация, описывающая определённую единицу (снова "ариергардный","шенграбенский"), которые не могут возводиться в множественное число.

Ещё одно отличие: в качестве синтаксических типов были выбраны  2 типа: прилагательное+существительное и глагол+ существительное. Довольно очевидно, что первая мера ассоциации не выдала пар второго образца, в остальных случаях по две пары попали в списки.

In [205]:
Counter(coll_mi).most_common(20)

[(('общежитейский', 'дело'), 11.83038763883099),
 (('душеспасительный', 'дело'), 11.83038763883099),
 (('каменьщический', 'дело'), 11.83038763883099),
 (('повестись', 'дело'), 11.83038763883099),
 (('островненский', 'дело'), 11.83038763883099),
 (('ариергардный', 'дело'), 11.83038763883099),
 (('денежный', 'дело'), 11.119894256025976),
 (('шенграбенский', 'дело'), 11.093422044664784),
 (('бедовый', 'дело'), 10.83038763883099),
 (('капитальный', 'дело'), 10.83038763883099),
 (('разлюбезный', 'дело'), 10.83038763883099),
 (('узнаться', 'дело'), 10.83038763883099),
 (('непредвиденный', 'дело'), 10.83038763883099),
 (('практический', 'дело'), 10.306825682773978),
 (('опекунский', 'дело'), 10.245425138109834),
 (('земледельческий', 'дело'), 10.245425138109834),
 (('испанский', 'дело'), 10.023032716773386),
 (('будничный', 'дело'), 9.83038763883099),
 (('этакий', 'дело'), 9.83038763883099),
 (('финансовый', 'дело'), 9.83038763883099)]

In [191]:
Counter(coll_t).most_common(20)

[(('свой', 'дело'), 7.125755063130326),
 (('весь', 'дело'), 6.973416449367931),
 (('самый', 'дело'), 5.756356128644291),
 (('другой', 'дело'), 4.671925676883667),
 (('чистый', 'дело'), 4.232673263216997),
 (('общий', 'дело'), 4.098197238779184),
 (('ваш', 'дело'), 4.072423052398192),
 (('мой', 'дело'), 4.042919267987911),
 (('важный', 'дело'), 3.9793364486099367),
 (('тот', 'дело'), 3.9730407267306296),
 (('иметь', 'дело'), 3.9305951812114492),
 (('быть', 'дело'), 3.5581651957554925),
 (('этот', 'дело'), 3.4386317828696376),
 (('денежный', 'дело'), 3.3151344871933976),
 (('наш', 'дело'), 3.2578406100764306),
 (('какой', 'дело'), 3.2320086886017285),
 (('такой', 'дело'), 3.1728933298423145),
 (('государственный', 'дело'), 3.156372830517938),
 (('практический', 'дело'), 2.82619416289795),
 (('странный', 'дело'), 2.80580625037054)]